# Get KEGG drug names data

In [120]:
import pandas as pd

#From http://www.kegg.jp/kegg/drug/

# problems with their list:
#    'Formaldehyde (USP); Formalin (JP17)' - this is in BNF but no INN or BAN
#    'Oxygen (JP17/USP)' - this is in BNF but no INN or BAN
# So - sometimes there is a BNF entry for things with no INN

name_systems = {
 'TN': 'Trade Name (example)',
 'JAN': 'Japanese Approved Name',
 'USAN': 'United States Adopted Name',
 'NF': 'National Formulary drug name',
 'INN': 'International Nonproprietary Name',
 'BAN': 'British Approved Name',
 'DCF': 'Dénomination commune française',
 'JP17': 'Japanese Pharmacopoeia, 17th edition',
 'USP': 'United States Pharmacopeia'}

url = 'http://rest.kegg.jp/list/drug'
data = pd.read_table(url, index_col=0, names = ['names'], header=0, sep="\t")
data = data[~pd.isnull(data['names'])]  # it includes nulls, for some reason

## Extract various country-specific names to their own columns

In [148]:
import re

def foo(field, requested_system):
    if not pd.isnull(field):
        for alternative in field.split(";"):
            match = re.match(r'^(.*) \((.*)\)', alternative)
            if not match:
                # There are usually only INNs for the moiety rather than the salt, etc.
                # print "oops", alternative, "###", field
                continue
            name, systems = match.groups()
            for system in systems.split('/'):
                if system == requested_system:
                    return name
    return None
data['INN'] = data['names'].apply(lambda x: foo(x, 'INN'))
data['BAN'] = data['names'].apply(lambda x: foo(x, 'BAN'))
data['USAN'] = data['names'].apply(lambda x: foo(x, 'USAN'))

data['USP'] = data['names'].apply(lambda x: foo(x, 'USP'))
data['non-BAN'] = data['names'].apply(lambda x: foo(x, 'USP') or foo(x, 'USAN') or foo(x, 'INN')).str.upper()
data.head()
data[data['names'].str.contains('etamine')]

,names,INN,BAN,USAN,USP,non-BAN
dr:D00711,Ketamine hydrochloride (JP17/USP); Ketalar (TN),None,None,None,Ketamine hydrochloride,KETAMINE HYDROCHLORIDE
dr:D01411,Lefetamine hydrochloride (JAN),None,None,None,None,None
dr:D01570,Iofetamine hydrochloride I 123 (USAN); N-Isopr...,None,None,Iofetamine hydrochloride I 123,None,IOFETAMINE HYDROCHLORIDE I 123
dr:D02074,Amphetamine sulfate (USP); Benzedrine (TN),None,None,None,Amphetamine sulfate,AMPHETAMINE SULFATE
dr:D02078,Dextroamphetamine sulfate (USP); Dexedrine (TN...,None,None,None,Dextroamphetamine sulfate,DEXTROAMPHETAMINE SULFATE
dr:D02242,Methamphetamine hydrochloride (JP17/USP); Deso...,None,None,None,Methamphetamine hydrochloride,METHAMPHETAMINE HYDROCHLORIDE
dr:D02291,Levmetamfetamine (USP/INN),Levmetamfetamine,None,None,Levmetamfetamine,LEVMETAMFETAMINE
dr:D02803,Aletamine hydrochloride (USAN),None,None,Aletamine hydrochloride,None,ALETAMINE HYDROCHLORIDE
dr:D02841,Altretamine (USP/INN); Hexalen (TN),Altretamine,None,None,Altretamine,ALTRETAMINE
dr:D03740,Dextroamphetamine (USAN); Dexamfetamine (INN),Dexamfetamine,None,Dextroamphetamine,None,DEXTROAMPHETAMINE


# Get FDA data

In [151]:
import requests
import subprocess
# from https://www.fda.gov/drugs/informationondrugs/ucm079750.htm#download
url = 'https://www.fda.gov/downloads/Drugs/InformationOnDrugs/UCM527389.zip'
r = requests.get(url, stream=True)
chunk_size = 2**16
with open('/tmp/pdf.zip', 'wb') as fd:
    for chunk in r.iter_content(chunk_size):
        fd.write(chunk)
subprocess.check_call('unzip /tmp/pdf.zip -d data/', shell=True)


0

## Turn it into just a list of ingredients

In [125]:
products_df = pd.read_table('data/Products.txt', sep="\t(?!$)")
all_us_ingredients = pd.DataFrame(
    products_df[~products_df.ActiveIngredient.str.contains(";")].ActiveIngredient.unique(),
    columns=['name'])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


## Merge FDA ingredients with KEGG names

In [126]:
asd = all_us_ingredients.merge(data, left_on='name', right_on='non-BAN', how='inner')
asd.head()

,name,names,INN,BAN,USAN,USP,non-BAN
0,HYDROXYAMPHETAMINE HYDROBROMIDE,Hydroxyamphetamine hydrobromide (USP); Paredri...,None,None,None,Hydroxyamphetamine hydrobromide,HYDROXYAMPHETAMINE HYDROBROMIDE
1,SULFAPYRIDINE,Sulfapyridine (USP/INN); Sulfapyridine (TN),Sulfapyridine,None,None,Sulfapyridine,SULFAPYRIDINE
2,HEPARIN SODIUM,Heparin sodium (JP17/USP/INN); Parnaparin sodi...,Heparin sodium,None,None,Heparin sodium,HEPARIN SODIUM
3,HISTAMINE PHOSPHATE,Histamine phosphate (USP); Histamine phosphate...,None,None,None,Histamine phosphate,HISTAMINE PHOSPHATE
4,BUTABARBITAL SODIUM,Butabarbital sodium (USP); Secbutabarbital sod...,Secbutabarbital sodium,None,None,Butabarbital sodium,BUTABARBITAL SODIUM


In [137]:
bnf_df = pd.DataFrame(
    pd.read_csv('data/bnf_codes.csv', 
                usecols=['BNF Chemical Substance'])['BNF Chemical Substance'].str.upper().unique(), 
    columns=['bnf_name'])

In [138]:
bnf_df.head()

,bnf_name
0,ALEXITOL SODIUM
1,ALMASILATE
2,ALUMINIUM HYDROXIDE
3,ALUMINIUM HYDROXIDE WITH MAGNESIUM
4,HYDROTALCITE


In [140]:
us_only = asd.merge(bnf_df, left_on='non-BAN', right_on='bnf_name', how='right')

In [141]:
us_only.head()

,name,names,INN,BAN,USAN,USP,non-BAN,bnf_name
0,SULFAPYRIDINE,Sulfapyridine (USP/INN); Sulfapyridine (TN),Sulfapyridine,None,None,Sulfapyridine,SULFAPYRIDINE,SULFAPYRIDINE
1,HEPARIN SODIUM,Heparin sodium (JP17/USP/INN); Parnaparin sodi...,Heparin sodium,None,None,Heparin sodium,HEPARIN SODIUM,HEPARIN SODIUM
2,AMINOPHYLLINE,Aminophylline (USP/INN); Somophyllin (TN); The...,Aminophylline,None,None,Aminophylline,AMINOPHYLLINE,AMINOPHYLLINE
3,METHYLTESTOSTERONE,Methyltestosterone (JP17/USP/INN); Android (TN...,Methyltestosterone,None,None,Methyltestosterone,METHYLTESTOSTERONE,METHYLTESTOSTERONE
4,ERGOCALCIFEROL,Ergocalciferol (JP17/USP); Vitamin D2; Calcife...,None,None,None,Ergocalciferol,ERGOCALCIFEROL,ERGOCALCIFEROL


In [150]:
unmatched = us_only[pd.isnull(us_only['non-BAN'])]
unmatched[~unmatched.bnf_name.str.contains('DUMMY') & ~unmatched.bnf_name.str.contains('OTHER')]['bnf_name'].head(50)

958                              ALEXITOL SODIUM
959                                   ALMASILATE
960                          ALUMINIUM HYDROXIDE
961           ALUMINIUM HYDROXIDE WITH MAGNESIUM
962                                 HYDROTALCITE
963                          MAGNESIUM CARBONATE
964     CO-MAGALDROX(MAGNESIUM/ALUMINIUM HYDROX)
965                              MAGNESIUM OXIDE
966                        MAGNESIUM TRISILICATE
967       ALUMINIUM & MAGNESIUM & ACT SIMETICONE
968                                   MAGALDRATE
969           ALUMINIUM & MAGNESIUM & OXETACAINE
970       CO-SIMALCITE (SIMETICONE/HYDROTALCITE)
971                          MAGNESIUM HYDROXIDE
972                                   SIMETICONE
973               CALCIUM CARBONATE & SIMETICONE
974                           MAGNESIUM SULPHATE
975                               SODIUM CITRATE
977                               GRIPE MIXTURES
979           ALGINIC ACID COMPOUND PREPARATIONS
980                 

This list demonstrates how hard the matching is likely to be. For example, it includes:

* CALCIUM CARBONATE which is not listed as a drug in the FDA database but is an "active ingredient" in other drugs
* SIMETICONE which is called SIMETHICONE in FDA database and is similarly not listed on its own
* KETAMINE which is listed as KETAMINE HCL in the FDA database
